In [1]:
import pandas as pd

In [2]:
import os

In [3]:
import glob

In [4]:
import re

In [5]:
pattern = re.compile(
    r'../../Hobotnica-GPU/results/GeneratedDenseBatch_size/(?P<cells>\d+)_cells_(?P<features>\d+)_features/(?P<batch_size>\d+)/_(?P<method>\w+)_(?P<metric>\w+).csv'
)

In [6]:
import numpy as np

In [7]:
results = []

for filename in glob.glob('../../Hobotnica-GPU/results/GeneratedDenseBatch_size/*_cells_*features/*/*_*_*.csv'):
    print(filename)
    group = pattern.match(filename)
    
    cells = int(group.group('cells'))
    features = int(group.group('features'))
    method = group.group('method')
    metric = group.group('metric')
    batch_size = int(group.group('batch_size'))
    
#     print(cells, features, method, metric)
    df = pd.read_csv(filename)
    
    elements_reinit = df.loc[:1, df.columns[0]]
    elements_sequential = df.loc[:, df.columns[0]]
    
    values = {
        'DataFile': filename,
        'metric': metric,
        'method': method,
        'wX': cells,
        'wY': features,
        'ID': f'{cells}_{features}',
        '|W|': cells * features,
        'logWX': np.log10(cells),
        'logWY': np.log10(features),
        'logMeanTime': np.log10(np.mean(elements_sequential)),
        'logMeanTimeReinit': np.log10(np.mean(elements_reinit)),
        'logMinTime': np.log10(np.min(elements_sequential)),
        'logMaxTime': np.log10(np.max(elements_sequential)),
        'minTime': np.min(elements_sequential),
        'maxTime': np.max(elements_sequential),
        'mean_reinit': np.mean(elements_reinit),
        'mean_sequential': np.mean(elements_sequential),
        'sd': np.std(elements_sequential),
        'Method': f'{metric}_{method}',
        'batch_size': batch_size,
        'num_batches': cells / batch_size,
    }
    
    for i in range(df.shape[0]):
        val = df.iloc[i][df.columns[0]]
        if val > 0:
            values[f'val_{i + 1}'] = df.iloc[i][df.columns[0]]
        
    results.append(values)

../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/1/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/50/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/5/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/20/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/10/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/100_cells_10_features/100/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/1000_cells_10_features/500/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/1000_cells_10_features/1000/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/1000_cells_10_features/50/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseBatch_size/1000_cells_10_features/200/_GPU_kendall.csv
../../Hobotnica-GPU/results/GeneratedDenseB

In [8]:
final_df = pd.DataFrame(results)

In [9]:
final_df.columns

Index(['DataFile', 'metric', 'method', 'wX', 'wY', 'ID', '|W|', 'logWX',
       'logWY', 'logMeanTime', 'logMeanTimeReinit', 'logMinTime', 'logMaxTime',
       'minTime', 'maxTime', 'mean_reinit', 'mean_sequential', 'sd', 'Method',
       'batch_size', 'num_batches', 'val_1', 'val_2', 'val_3', 'val_4',
       'val_5', 'val_6', 'val_7', 'val_8', 'val_9', 'val_10'],
      dtype='object')

In [10]:
final_df.to_csv('../data/generated_dense_batch_size.csv', index=None)